In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import visualkeras

In [2]:
matches = pd.read_csv("upcoming.csv", index_col=0)

In [3]:
matches['over_under'] = ((matches['gf'] + matches['ga']) > 2.5).astype("int")

In [4]:
#Encoding label we make result being a numeric value, which we can predict later. Instead of W, D, L.
yle = LabelEncoder()
matches['over_under'] = yle.fit_transform(matches['over_under'])

In [5]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,g-xg,npxg/sh,season,team,over_under
1,2022-08-06,15:30,Bundesliga,Matchweek 1,Sat,Home,W,3.0,1.0,Hertha BSC,...,Match Report,NaN,19.0,8.0,18.5,2.1,0.05,2023,Union Berlin,1
2,2022-08-14,15:30,Bundesliga,Matchweek 2,Sun,Away,D,0.0,0.0,Mainz 05,...,Match Report,NaN,10.0,2.0,21.5,-0.4,0.04,2023,Union Berlin,0
3,2022-08-20,18:30,Bundesliga,Matchweek 3,Sat,Home,W,2.0,1.0,RB Leipzig,...,Match Report,NaN,7.0,2.0,16.8,1.5,0.07,2023,Union Berlin,1
4,2022-08-27,15:30,Bundesliga,Matchweek 4,Sat,Away,W,6.0,1.0,Schalke 04,...,Match Report,NaN,12.0,7.0,15.3,4.7,0.11,2023,Union Berlin,1
5,2022-09-03,15:30,Bundesliga,Matchweek 5,Sat,Home,D,1.0,1.0,Bayern Munich,...,Match Report,NaN,7.0,2.0,23.5,0.7,0.05,2023,Union Berlin,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-14,15:30,Bundesliga,Matchweek 30,Sat,Away,L,1.0,2.0,Hertha BSC,...,Match Report,NaN,13.0,2.0,14.2,-0.6,0.13,2018,Koln,1
39,2018-04-22,18:00,Bundesliga,Matchweek 31,Sun,Home,D,2.0,2.0,Schalke 04,...,Match Report,NaN,11.0,3.0,19.6,1.1,0.08,2018,Koln,1
40,2018-04-28,15:30,Bundesliga,Matchweek 32,Sat,Away,L,2.0,3.0,Freiburg,...,Match Report,NaN,13.0,2.0,16.2,0.6,0.10,2018,Koln,1
41,2018-05-05,15:30,Bundesliga,Matchweek 33,Sat,Home,L,1.0,3.0,Bayern Munich,...,Match Report,NaN,10.0,3.0,13.0,-1.7,0.17,2018,Koln,1


In [6]:
#Using pandas sweets.
matches["date"] = pd.to_datetime(matches["date"])

#Creating numeric values for Home and Away, so that machine can learn from it.
matches['venue_c'] = matches['venue'].astype('category').cat.codes

#Creating unique code for each opponent squad.
matches['opp_c'] = matches['opponent'].astype('category').cat.codes

#Creating a normalized hour time.
matches['hour'] = matches['time'].str.replace(":.+", "", regex=True).astype("int")

#Monday = 0, Tuesday = 1, etc.
matches['day_c'] = matches['date'].dt.dayofweek

matches = matches.drop(columns=['round', 'day', 'comp'])

In [7]:
matches = matches[['date', 'result', 'venue', 'gf', 'ga', 'xg', 'xga',
       'poss', 'sh', 'sot', 'dist', 'g-xg', 'team', 'opponent', 'opp_c', 'over_under']]

matches = matches.replace("Bayer Leverkusen", "Leverkusen")
matches = matches.replace("Eintracht Frankfurt", "Eint Frankfurt")
matches = matches.replace("M'Gladbach", "Monchengladbach")


In [8]:
at_home = matches['venue'] == "Home"
matches_home = matches[at_home].sort_values(['team','date'])
matches_home['xg_ha'] = matches_home['xg'].rolling(38, closed='left').mean()
matches_home['xga_ha'] = matches_home['xga'].rolling(38, closed='left').mean()
matches_home['g-xg_ha'] = matches_home['g-xg'].rolling(38, closed='left').mean()

In [9]:
at_away = matches['venue'] == "Away"
matches_away = matches[at_away].sort_values(['team', 'date'])
matches_away['xg_ha'] = matches_away['xg'].rolling(38, closed='left').mean()
matches_away['xga_ha'] = matches_away['xga'].rolling(38, closed='left').mean()
matches_away['g-xg_ha'] = matches_away['g-xg'].rolling(38, closed='left').mean()

In [10]:
c = [matches_home, matches_away]

In [11]:
grouping = pd.concat(c)

In [12]:
matches = grouping.round(2)

In [13]:
is_home = matches['venue'] == "Home"
home = matches[is_home]
away = matches[~is_home]

In [14]:
#Setting up our data, so we can add opponents value at the current time to our DataFrame.
home = home.merge(
    away[["date", "opponent", "team", "xg", "xga", "sh", "poss", "g-xg", "xg_ha", "xga_ha", "g-xg_ha"]],
    left_on=["date", "team", "opponent"],
    right_on=["date", "opponent", "team"],
    how="outer",
    suffixes=("", "_opp"),
).drop(columns=["opponent_opp", "team_opp"])

In [15]:
home = home.dropna()
matches = home

In [16]:
grouped_matches = matches.groupby('team')

In [17]:
group = grouped_matches.get_group("Stuttgart")

In [18]:
#Short function to get rolling averages as our predictors from last 3 matches.
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(35, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [19]:
#cols = ['poss', 'sh', 'xg', 'xga', 'g-xg', 'poss_opp', 'sh_opp', 'xg_opp', 'xga_opp', 'g-xg_opp']
predictors = ['xg_ha', 'xga_ha', 'xg_ha_opp', 'xga_ha_opp', 'g-xg_ha', 'g-xg_ha_opp']
cols = ['xg', 'xga', 'g-xg', 'xg_opp', 'xga_opp', 'g-xg_opp']
new_cols = [f"{c}R" for c in cols]

In [20]:
#Getting the rolling averages.
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

In [21]:
df = matches_rolling
df = df.sort_values('date')

In [22]:
#Setting up our neural network model with 12 input layers, a lot of hidden layers and 3 output layers (Win, Draw, Lose). 

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(12,)),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(2, activation="softmax")
])

In [23]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
#Setting up training and testing slices.
train = df[df['date'] <= '2021-10-04']
test = df[df['date'] > '2021-10-04']

In [25]:
model.fit(train[predictors + new_cols], train['over_under'], epochs=150)

Epoch 1/150
13/13 [==============================] - 0s 1ms/step - loss: 0.7130 - accuracy: 0.6241
Epoch 2/150
13/13 [==============================] - 0s 1ms/step - loss: 0.6766 - accuracy: 0.6241
Epoch 3/150
13/13 [==============================] - 0s 1ms/step - loss: 0.6661 - accuracy: 0.6241
Epoch 4/150
13/13 [==============================] - 0s 1ms/step - loss: 0.6657 - accuracy: 0.6241
Epoch 5/150
13/13 [==============================] - 0s 1ms/step - loss: 0.6643 - accuracy: 0.6241
Epoch 6/150
13/13 [==============================] - 0s 2ms/step - loss: 0.6636 - accuracy: 0.6241
Epoch 7/150
13/13 [==============================] - 0s 1ms/step - loss: 0.6650 - accuracy: 0.6241
Epoch 8/150
13/13 [==============================] - 0s 1ms/step - loss: 0.6638 - accuracy: 0.6241
Epoch 9/150
13/13 [==============================] - 0s 2ms/step - loss: 0.6627 - accuracy: 0.6241
Epoch 10/150
13/13 [==============================] - 0s 2ms/step - loss: 0.6623 - accuracy: 0.6241
Epoch 11/

In [26]:
test_loss, test_acc = model.evaluate(test[predictors + new_cols], test['over_under'])

13/13 [==============================] - 0s 1ms/step - loss: 0.6348 - accuracy: 0.6748


In [27]:
prediction = model.predict(test[predictors + new_cols])

13/13 [==============================] - 0s 1ms/step


In [28]:
prediction

array([[0.360801  , 0.639199  ],
       [0.41635945, 0.5836405 ],
       [0.3641358 , 0.6358642 ],
       [0.34129575, 0.6587043 ],
       [0.40480492, 0.59519506],
       [0.45387033, 0.54612964],
       [0.28541127, 0.7145887 ],
       [0.19450022, 0.80549973],
       [0.37700087, 0.6229991 ],
       [0.4498995 , 0.5501005 ],
       [0.42403138, 0.5759687 ],
       [0.43659297, 0.56340706],
       [0.3796922 , 0.62030786],
       [0.37908328, 0.6209167 ],
       [0.4643565 , 0.5356435 ],
       [0.3571454 , 0.64285463],
       [0.38698608, 0.613014  ],
       [0.35904405, 0.640956  ],
       [0.16282561, 0.8371744 ],
       [0.360258  , 0.639742  ],
       [0.42047063, 0.5795294 ],
       [0.38666457, 0.61333543],
       [0.44259912, 0.5574008 ],
       [0.31241587, 0.6875841 ],
       [0.3204917 , 0.6795082 ],
       [0.44258618, 0.5574138 ],
       [0.4545904 , 0.5454096 ],
       [0.43075132, 0.5692486 ],
       [0.43652686, 0.5634731 ],
       [0.3324064 , 0.66759354],
       [0.

In [29]:
#Adding the probabilities to our sweet DataFrame.
test['Over'] = prediction[:,1]
test['Under'] = prediction[:,0]

C:\Users\PC\AppData\Local\Temp\ipykernel_17068\173245236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Over'] = prediction[:,1]
C:\Users\PC\AppData\Local\Temp\ipykernel_17068\173245236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Under'] = prediction[:,0]


In [30]:
test = test[['date', 'team', 'opponent', 'Over', 'Under']].round(2)

In [31]:
test['Over'] = 1/test['Over']
test['Under'] = 1/test['Under']

In [32]:
test = test.sort_values('date').round(2)

In [35]:
test.round(2)[15:45]

,date,team,opponent,Over,Under
473,2021-11-05,Mainz 05,Monchengladbach,1.56,2.78
791,2021-11-06,Wolfsburg,Augsburg,1.64,2.56
585,2021-11-06,RB Leipzig,Dortmund,1.56,2.78
83,2021-11-06,Bayern Munich,Freiburg,1.19,6.25
305,2021-11-07,Hertha BSC,Leverkusen,1.56,2.78
27,2021-11-19,Augsburg,Bayern Munich,1.72,2.38
140,2021-11-20,Dortmund,Stuttgart,1.45,3.23
418,2021-11-20,Leverkusen,Bochum,1.64,2.56
693,2021-11-20,Union Berlin,Hertha BSC,1.79,2.27
361,2021-11-20,Hoffenheim,RB Leipzig,1.47,3.12


In [34]:
#Schalke 04, Werder Bremen